In [1]:
#%pip install SQLAlchemy pymysql python-dotenv 

# 1. Connect to SQL Database


- It would be bad practice to store your database password into a Python script or Jupyter Notebook
- Instead, we store the password in a separate `.env` file.
- This is a text file that contains environment variables in the form of name=value pairs.
- You can then load these variables into environment variables using the dotenv package.
- Such a .env file should never be shared with others or checked into version control (will be covered in our lecture on git and Github)


In [2]:
from dotenv import load_dotenv                        # Load passwords etc from .env file 
load_dotenv('.env')                                   # Absolute or relative path to the .env file 

True

- The package SQLAlchemy allows you to connect to different variants of SQL databases (e.g. MySQL, PostgreSQL, SQLite, ...).
- To connect to a MySQL database, you need to specify both the database dialect (mysql) and choose an appropriate driver (pymysql or mysqlconnector).


In [3]:
from sqlalchemy import create_engine           # Connection to database
import os                                             # Needed to access environment variables

In [4]:
DIALECT = 'mysql'
DRIVER = 'mysqlconnector'
USER = 'root'
PASSWORD = os.getenv('MYSQL_PASSWORD')
HOST = 'localhost'
PORT = '3306'
DB = 'music'

connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}"
engine = create_engine(connection_string)

# 2. Read from SQL Database to Pandas DataFrame


## 2.1 Inspecting the MySQL database


First, we inspect the available databases in our MySQL Server, and specifically the available tables in our "music" database.


In [5]:
import pandas as pd
from sqlalchemy import inspect

In [6]:
inspector = inspect(engine)

print('Available Databases:')
inspector.get_schema_names()

Available Databases:


['Sales',
 'University',
 'information_schema',
 'music',
 'mysql',
 'performance_schema',
 'sys']

In [7]:
inspector.get_table_names()

['artists', 'avg_streams_by_region', 'charts', 'tracks', 'tracks_artists']

## 2.2 Read full table


We can read entire tables into a Pandas Dataframe


In [8]:
tracks = pd.read_sql('charts', con=engine)
tracks.head()

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0
2,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0
3,1,de,weekly,2021-06-03,2knDuFWczh1Wv92eBAdPTa,4,2739815,1,6.0
4,1,de,weekly,2021-06-03,3idbCPumlga23rxFA08fZE,5,2664580,5,1.0


## 2.3 SQL Select Query


We can send arbitrary SQL queries to read data into a Pandas DataFrame


### Select all columns


In [9]:
query = "select * from charts limit 3"
pd.read_sql(query, con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0
2,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0


### Where conditions


In [10]:
query = "select * from charts where streams >  17000000"
pd.read_sql(query, con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,66,us,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,24777068,1,3.0
1,67,us,weekly,2021-06-10,4ZtFanR9U6ndgddUvNcjcG,1,20688190,1,4.0
2,68,us,weekly,2021-06-17,4ZtFanR9U6ndgddUvNcjcG,1,17136875,1,5.0


### Aggregations


In [11]:
query = "select count(*), avg(streams) from charts "
pd.read_sql(query, con=engine)

,count(*),avg(streams)
0,15598,1.030511e+06


### Grouped aggregations


In [12]:
query = "select region, avg(streams) from charts group by region order by avg(streams) desc"
pd.read_sql(query, con=engine)

,region,avg(streams)
0,us,2.606887e+06
1,de,8.139232e+05
2,es,7.866070e+05
3,it,7.221330e+05
4,gb,6.968614e+05
5,fr,5.564647e+05


### Joining tables


In [13]:
query = """select * from charts
           left join tracks
           on charts.track_id = tracks.id"""
pd.read_sql(query, con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart,id,...,mode,tempo,explicit,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0,4ZtFanR9U6ndgddUvNcjcG,...,mayor,166.928,1,0.563,0.664,0.1540,0.3350,0.000000,0.0849,0.688
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0,6DiaHETMcKdX7FwuGXwp1O,...,mayor,94.915,1,0.841,0.871,0.0585,0.5440,0.000000,0.0853,0.564
2,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0,4txsSBdT4yVJtUj41YaQEB,...,mayor,102.022,1,0.959,0.524,0.1830,0.0686,0.000000,0.0913,0.285
3,1,de,weekly,2021-06-03,2knDuFWczh1Wv92eBAdPTa,4,2739815,1,6.0,2knDuFWczh1Wv92eBAdPTa,...,mayor,82.009,1,0.743,0.601,0.0615,0.3360,0.000002,0.0902,0.445
4,1,de,weekly,2021-06-03,3idbCPumlga23rxFA08fZE,5,2664580,5,1.0,3idbCPumlga23rxFA08fZE,...,mayor,149.610,0,0.468,0.669,0.1550,0.0572,0.000057,0.1370,0.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15593,78,us,weekly,2021-08-26,38GBNKZUhfBkk3oNlWzRYd,196,1450097,13,7.0,38GBNKZUhfBkk3oNlWzRYd,...,mayor,85.016,1,0.765,0.373,0.0713,0.3410,0.525000,0.1120,0.554
15594,78,us,weekly,2021-08-26,7fBv7CLKzipRk6EC6TWHOB,197,1446058,27,41.0,7fBv7CLKzipRk6EC6TWHOB,...,minor,113.003,1,0.585,0.564,0.0515,0.0671,0.000000,0.1350,0.137
15595,78,us,weekly,2021-08-26,7FIWs0pqAYbP91WWM0vlTQ,198,1445715,2,81.0,7FIWs0pqAYbP91WWM0vlTQ,...,minor,165.995,1,0.808,0.745,0.3420,0.1450,0.000000,0.2920,0.829
15596,78,us,weekly,2021-08-26,07MDkzWARZaLEdKxo6yArG,199,1442195,199,1.0,07MDkzWARZaLEdKxo6yArG,...,mayor,94.995,1,0.773,0.470,0.0299,0.0153,0.000193,0.0851,0.399


# 3. Write Pandas DataFrame to SQL Database


For this demo, we create some new DataFrame with aggregated data, and then write this data back to the database


In [14]:
query = """select region, avg(streams) as avg_streams from charts
           left join tracks
           on charts.track_id = tracks.id 
           group by region
           order by avg_streams desc"""
aggregated_data = pd.read_sql(query, con=engine)
aggregated_data

,region,avg_streams
0,us,2.606887e+06
1,de,8.139232e+05
2,es,7.866070e+05
3,it,7.221330e+05
4,gb,6.968614e+05
5,fr,5.564647e+05


In [15]:
aggregated_data.to_sql(name="avg_streams_by_region", 
                       con=engine, 
                       if_exists='replace',   # fail, replace, append 
                       index=False)

6

In [16]:
inspect(engine).get_table_names()

['artists', 'avg_streams_by_region', 'charts', 'tracks', 'tracks_artists']

# 4. Execute Arbitrary SQL


So far, we have only used Pandas `read_sql` and `write_sql` functions to interact with our database. **However, we can run arbitrary SQL from our Python session**.

In the following we:

1. Connect to our MySQL Server
2. We create a new test database
3. We create a new table from a Pandas DataFrame
4. We clean up by dropping the table and the database


In [17]:
from sqlalchemy import text
import seaborn as sns

### Connect to MySQL Server


Here we do not connect to a specific database, but to the MySQL Server (one level higher in the hierarchy). This will allow us to manage (create, alter, delete ...) entire databases


In [18]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}"
server_engine = create_engine(connection_string)

### Create a new database


In [19]:
with server_engine.connect() as connection:
    connection.execute(text('DROP DATABASE IF EXISTS test'))
    connection.execute(text('CREATE DATABASE test'))

In [20]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/test"
test_engine = create_engine(connection_string)

In [21]:
titanic = sns.load_dataset('titanic')
titanic.to_sql(name='titanic', con=test_engine)

891

### Clean up: drop table and database


In [22]:
with server_engine.connect() as connection:
    connection.execute(text("Drop table if exists test.titanic"))
    connection.execute(text("Drop database if exists test"))

In [ ]:
artist 

NameError: name 'artist' is not defined

In [32]:
tracks_artists

,track_id,artist_id
0,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W
1,1xK1Gg9SxG8fy2Ya373oqb,2LmcxBak1alK1bf7d1beTr
2,1uXJYnCsXy2v50y0fDEEiV,7gjqZ8coFZimZDtdk04WP1
3,1uXJYnCsXy2v50y0fDEEiV,1MRiIeZbc0cRuxOafDUCtH
4,1uXJYnCsXy2v50y0fDEEiV,3hBQ4zniNdQf1cqqo6hzuW
...,...,...
3580,67qxSUoyng1ohzvfyOyNWW,1k3zqz5SgrYCE0wpcfJX3g
3581,1t0nF0d3wzsn0SqgtUGhRA,2K5nCggbhSZ00YCYP5qkZS
3582,6KF78W66DcQ9m5YqR0r5Ig,3pgCLfNbw5ozIfoNsvDU7i
3583,6KF78W66DcQ9m5YqR0r5Ig,2YVF0Ou5zIc4mpgtLIlGN0


In [23]:
query = "select * from tracks INNER JOIN tracks_artists ON tracks.id = tracks_artists.track_id INNER JOIN artists ON tracks_artists.artist_id = artists.id"

In [24]:
dfa = pd.read_sql(query, con=engine)

In [36]:
tracks = pd.read_sql("select * from tracks", con=engine)
tracks.head()
tracks.columns

Index(['id', 'name', 'duration', 'mode', 'tempo', 'explicit', 'danceability',
       'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence'],
      dtype='object')

In [35]:
artists = pd.read_sql("select * from artists", con=engine)
artists.head()
artists.columns
    

Index(['id', 'name', 'pop', 'rap', 'hip-hop', 'electronic', 'trap', 'rock',
       'indie', 'rnb', 'soul', 'house', 'latin', 'reggaeton',
       'singer-songwriter', 'jazz', 'dance', 'folk', 'alternative', 'country'],
      dtype='object')

In [29]:
dfa.columns

Index(['id', 'name', 'duration', 'mode', 'tempo', 'explicit', 'danceability',
       'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'track_id', 'artist_id', 'id', 'name', 'pop', 'rap',
       'hip-hop', 'electronic', 'trap', 'rock', 'indie', 'rnb', 'soul',
       'house', 'latin', 'reggaeton', 'singer-songwriter', 'jazz', 'dance',
       'folk', 'alternative', 'country'],
      dtype='object')

In [25]:
dfa.columns

Index(['id', 'name', 'duration', 'mode', 'tempo', 'explicit', 'danceability',
       'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'track_id', 'artist_id', 'id', 'name', 'pop', 'rap',
       'hip-hop', 'electronic', 'trap', 'rock', 'indie', 'rnb', 'soul',
       'house', 'latin', 'reggaeton', 'singer-songwriter', 'jazz', 'dance',
       'folk', 'alternative', 'country'],
      dtype='object')

Close all open connections, if present.


In [26]:
engine.dispose()
server_engine.dispose()
test_engine.dispose()